# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

In [1]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 3.8MB 39.8MB/s 
     |████████████████████████████████| 92kB 7.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing inst

# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [259]:
import requests
from bs4 import BeautifulSoup 

def get_news_by_url(url):
  res=requests.get(url)
  bs=BeautifulSoup(res.content, 'html.parser')

  title=bs.select('h3#articleTitle')[0].text
  content=bs.select('#articleBodyContents')[0].get_text().replace('\n'," ")
  content=content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return content.strip()

docs=[]
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 1) 전처리

In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [5]:
from sklearn.feature_extraction.text import CountVectorizer 

count_vectorizer=CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector=count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

### 3) 핵심키워드 추출

In [8]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [13]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
 
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.2526148007071733)
('계획', 0.21652697203472)
('정통부', 0.18043914336226666)
('과기', 0.18043914336226666)
('통해', 0.17469259767293158)


## 실습 2. gensim 활용


### 1) 전처리

In [244]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [16]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [17]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.260111262735105)
('폼', 0.260111262735105)
('계획', 0.2229525109158043)
('정통부', 0.18579375909650356)
('과기', 0.18579375909650356)




---



# 8-2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


### 1) 토큰화 (Tokenization)

분석 텍스트 정제

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [21]:
import pandas as pd
import numpy as np

In [ ]:
token = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = None #vocab을 인덱스로 변환
idx2vocab = None  #인덱스를 vocab으로 변환
vocab2idx

In [20]:
class vo_idx():
  def __init__(self,nod_list):
    self.nod_list=nod_list
  def vo_to_dic(self):
    dic_refer=list(set(self.nod_list))
    return {dic_refer[i]:i for i in range(len(dic_refer))}
  def idx_to_dic(self):
    dic_refer=list(set(self.nod_list))
    return {i:dic_refer[i] for i in range(len(dic_refer))}
  def vo_to_idx(self,vo_list):
    return [(self.vo_to_dic())[vo] if vo in list(self.vo_to_dic().keys()) else 'nan' for vo in vo_list]
  def idx_to_vo(self,num_list):
    return [(self.idx_to_dic())[nu] if nu in list(self.idx_to_dic().keys()) else 'nan' for nu in num_list]
  

In [ ]:
vo_idx(nodes).vo_to_dic()

{'딸기': 1, '바나나': 2, '사과': 0, '파인애플': 3}

In [ ]:
nodes

['바나나', '사과', '파인애플', '딸기']

In [ ]:
vo_idx(nodes).vo_to_idx(token)

[1, 2, 0, 1, 3]

In [ ]:
vo_idx(nodes).idx_to_vo([1,2,1])

['딸기', '바나나', '딸기']

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
vocab2idx=vo_idx(nodes).vo_to_dic()

In [ ]:
vocab2idx.items()

dict_items([('사과', 0), ('딸기', 1), ('바나나', 2), ('파인애플', 3)])

In [ ]:
vo_idx(nodes).vo_to_dic()

{'딸기': 1, '바나나': 2, '사과': 0, '파인애플': 3}

In [ ]:
[vo_idx(nodes).vo_to_dic()[tok] for tok in token]

[1, 2, 0, 1, 3]

In [ ]:
np.matrix([[i,k] for i,k in vocab2idx.items()])

matrix([['사과', '0'],
        ['딸기', '1'],
        ['파인애플', '2'],
        ['바나나', '3']], dtype='<U4')

In [ ]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.matrix([[0 for i in range(len(vo_idx(nodes).vo_to_dic()))] for i in range(len(vo_idx(nodes).vo_to_dic()))])

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((weighted_edge.shape[0],1))

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2

# weighted_edge 구현


weighted_edge

matrix([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])

In [ ]:
covered_coocurrences = weighted_edge
covered_coocurrences

matrix([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])

In [ ]:
for idx, tok in enumerate(token):
  if window_size-1>0:
    if len(token[idx:])>=window_size:
      for win in range(1,window_size):
        covered_coocurrences[vo_idx(nodes).vo_to_dic()[tok],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
        covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1
    else:
      for win in range(1,len(token[idx:])):
        covered_coocurrences[vo_idx(nodes).vo_to_dic()[tok],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
        covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1

In [ ]:
token

['딸기', '바나나', '사과', '딸기', '파인애플']

In [ ]:
vo_idx(nodes).vo_to_dic()

{'딸기': 1, '바나나': 2, '사과': 0, '파인애플': 3}

In [ ]:
covered_coocurrences

matrix([[0, 1, 1, 0],
        [1, 0, 1, 1],
        [1, 1, 0, 0],
        [0, 1, 0, 0]])

In [ ]:
weighted_edge=covered_coocurrences/np.matrix([cc.sum() for cc in covered_coocurrences]).transpose()

In [ ]:
weighted_edge

matrix([[0.        , 0.5       , 0.5       , 0.        ],
        [0.33333333, 0.        , 0.33333333, 0.33333333],
        [0.5       , 0.5       , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        ]])

In [ ]:
np.c_[weighted_edge[:,0]*score[0],weighted_edge[:,1]*score[1]]

matrix([[0.        , 0.5       ],
        [0.33333333, 0.        ],
        [0.5       , 0.5       ],
        [0.        , 1.        ]])

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [ ]:
constant_edge=np.copy(weighted_edge)

In [ ]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    for i in range(len(weighted_edge)):
      score[i]=(1-d)+d*((weighted_edge[:,i]).sum())
    for prev, curr in zip(prev_score,score):
      if abs(prev-curr)<threshold:
        break
    for j in range(len(weighted_edge)):
      weighted_edge[j,:]=constant_edge[j,:]*float(score[j])

In [ ]:
score

array([[0.98371127],
       [1.46694347],
       [0.98371127],
       [0.56563398]])

### 5) 핵심 단어 추출

In [ ]:
vo_idx(nodes).vo_to_dic()

{'딸기': 1, '바나나': 2, '사과': 0, '파인애플': 3}

In [ ]:
sorted_index = list(vo_idx(nodes).vo_to_dic().values())

n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(vo_idx(nodes).idx_to_dic()[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
사과 : [0.98371127]
딸기 : [1.46694347]
바나나 : [0.98371127]
파인애플 : [0.56563398]


---

In [56]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [57]:
from konlpy.tag import Mecab
mecab = Mecab()

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체

### 2) 그래프 생성 (weighted edge 계산)

In [65]:
import numpy as np
import math
vocab_len = len(nodes)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.matrix([[0 for i in range(len(vo_idx(nodes).vo_to_dic()))] for i in range(len(vo_idx(nodes).vo_to_dic()))])

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((weighted_edge.shape[0],1))

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 10

# weighted_edge 구현


weighted_edge

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [66]:
covered_coocurrences = weighted_edge
covered_coocurrences

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [121]:
def cc_maker(token):
  for idx, tok in enumerate(token):
    if window_size-1>0:
      if len(token[idx:])>=window_size:
        for win in range(1,window_size):
          if (tok in list(vo_idx(nodes).vo_to_dic().keys()))&(token[idx+win] in list(vo_idx(nodes).vo_to_dic().keys())):
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[(tok)],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1
      else:
        for win in range(1,len(token[idx:])):
          if (tok in list(vo_idx(nodes).vo_to_dic().keys()))&(token[idx+win] in list(vo_idx(nodes).vo_to_dic().keys())):
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[(tok)],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1

In [122]:
cc_maker(tokens)

In [123]:
weighted_edge=covered_coocurrences/np.matrix([cc.sum() for cc in covered_coocurrences]).transpose()

### 3) 각 노드의 score계산

In [124]:
constant_edge=np.copy(weighted_edge)

In [125]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    for i in range(len(weighted_edge)):
      score[i]=(1-d)+d*((weighted_edge[:,i]).sum())
    for prev, curr in zip(prev_score,score):
      if abs(prev-curr)<threshold:
        break
    for j in range(len(weighted_edge)):
      weighted_edge[j,:]=constant_edge[j,:]*float(score[j])

In [126]:
weighted_edge

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.11289877],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.10150759, ..., 0.        , 0.        ,
         0.        ]])

### 4)핵심 단어 추출

In [127]:
sorted_index = list(vo_idx(nodes).vo_to_dic().values())

n = 10

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(vo_idx(nodes).idx_to_dic()[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
('노웅래', 'NNP') : [0.95357835]
('마련', 'NNG') : [1.99385478]
('현실', 'NNG') : [0.56449387]
('전략', 'NNG') : [0.6488513]
('이재운', 'NNP') : [0.97072936]
('경제', 'NNG') : [2.96444605]
('중구', 'NNP') : [0.87802279]
('실제', 'NNG') : [0.7169071]
('사업', 'NNG') : [0.88331405]
('신규', 'NNG') : [1.0879946]


## 실습 3. TextRank 핵심 구 추출

In [172]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

### 1) 명사를 기준으로 구 추출

In [217]:
from konlpy.tag import Mecab
mecab = Mecab()

# nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP','NNB','NNBC','NP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)]

In [226]:
tokens=[]
gugus=[]
i=1
for idx, (token, taeso) in enumerate(mecab.pos(doc)):
 if i==1:
   if (taeso[0]=='N')&(taeso!='NNBC'):
    gugu=token
    i=1
    while mecab.pos(doc)[idx+i][1][0]=='N':
      gugu+=' '+mecab.pos(doc)[idx+i][0]
      i+=1
    gugus.append((gugu,'GUGU'))
    tokens.append((gugu,'GUGU'))
   else:
     tokens.append((token,taeso))
 else:
   i-=1

In [ ]:
#기존 코드를 활용하기 위해 gugus를 nodes에 넣어줌.
nodes=gugus

In [228]:
gugus

[('과기 정통부', 'GUGU'),
 ('유영민 장관 등 참석', 'GUGU'),
 ('기념행사', 'GUGU'),
 ('억 원 투입', 'GUGU'),
 ('여종 데이터 구축 민간 클라우드', 'GUGU'),
 ('외부 연계 체계', 'GUGU'),
 ('개방', 'GUGU'),
 ('강화', 'GUGU'),
 ('데일리 이재운 기자', 'GUGU'),
 ('국가 차원', 'GUGU'),
 ('빅 데이터 활용 시대', 'GUGU'),
 ('산업 창출', 'GUGU'),
 ('기존 산업', 'GUGU'),
 ('변화', 'GUGU'),
 ('혁신', 'GUGU'),
 ('장', 'GUGU'),
 ('센터', 'GUGU'),
 ('문', 'GUGU'),
 ('분야', 'GUGU'),
 ('데이터 경제', 'GUGU'),
 ('발전', 'GUGU'),
 ('정부', 'GUGU'),
 ('청사진', 'GUGU'),
 ('현실', 'GUGU'),
 ('구현', 'GUGU'),
 ('데', 'GUGU'),
 ('계획', 'GUGU'),
 ('과학 기술 정보 통신부', 'GUGU'),
 ('서울 중구 대한 상공 회의소', 'GUGU'),
 ('데이터 생태', 'GUGU'),
 ('조성', 'GUGU'),
 ('혁신 성장', 'GUGU'),
 ('기반 마련', 'GUGU'),
 ('빅 데이터 플랫 폼', 'GUGU'),
 ('센터', 'GUGU'),
 ('출범식 행사', 'GUGU'),
 ('개최', 'GUGU'),
 ('유영민 과기 정통부 장관', 'GUGU'),
 ('노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 등', 'GUGU'),
 ('참가', 'GUGU'),
 ('분야', 'GUGU'),
 ('센터', 'GUGU'),
 ('간', 'GUGU'),
 ('억 원 투입 이미지', 'GUGU'),
 ('픽사 베이 빅 데이터', 'GUGU'),
 ('데이터 활용', 'GUGU'),
 ('혁신', 'GUGU'),
 ('장', 'GUGU'),
 ('문재', 'GUG

In [229]:
tokens

[('과기 정통부', 'GUGU'),
 (',', 'SC'),
 ('22', 'SN'),
 ('일', 'NNBC'),
 ('유영민 장관 등 참석', 'GUGU'),
 ('해', 'XSV+EC'),
 ('기념행사', 'GUGU'),
 ('2021', 'SN'),
 ('년', 'NNBC'),
 ('까지', 'JX'),
 ('1516', 'SN'),
 ('억 원 투입', 'GUGU'),
 (',', 'SC'),
 ('5100', 'SN'),
 ('여종 데이터 구축 민간 클라우드', 'GUGU'),
 ('통한', 'VV+ETM'),
 ('외부 연계 체계', 'GUGU'),
 ('도', 'JX'),
 ('.', 'SF'),
 ('."', 'SY'),
 ('개방', 'GUGU'),
 ('성', 'XSN'),
 ('강화', 'GUGU'),
 ('"[', 'SY'),
 ('이', 'JKS'),
 ('데일리 이재운 기자', 'GUGU'),
 (']', 'SSC'),
 ('국가 차원', 'GUGU'),
 ('의', 'JKG'),
 ('빅 데이터 활용 시대', 'GUGU'),
 ('가', 'JKS'),
 ('열린다', 'VV+EF'),
 ('.', 'SF'),
 ('새로운', 'VA+ETM'),
 ('산업 창출', 'GUGU'),
 ('과', 'JC'),
 ('기존 산업', 'GUGU'),
 ('의', 'JKG'),
 ('변화', 'GUGU'),
 ('에', 'JKB'),
 ('이르', 'VV'),
 ('는', 'ETM'),
 ('‘', 'SY'),
 ('혁신', 'GUGU'),
 ('성', 'XSN'),
 ('장', 'GUGU'),
 ('’', 'SY'),
 ('을', 'JKO'),
 ('위한', 'VV+ETM'),
 ('센터', 'GUGU'),
 ('가', 'JKS'),
 ('문', 'GUGU'),
 ('을', 'JKO'),
 ('연다', 'VV+EC'),
 ('.', 'SY'),
 ('10', 'SN'),
 ('개', 'NNBC'),
 ('분야', 'GUGU'),
 ('에'

### 2) 각 구의 Score 계산

In [230]:
nodes=gugus

In [231]:
nodes[:10]

[('과기 정통부', 'GUGU'),
 ('유영민 장관 등 참석', 'GUGU'),
 ('기념행사', 'GUGU'),
 ('억 원 투입', 'GUGU'),
 ('여종 데이터 구축 민간 클라우드', 'GUGU'),
 ('외부 연계 체계', 'GUGU'),
 ('개방', 'GUGU'),
 ('강화', 'GUGU'),
 ('데일리 이재운 기자', 'GUGU'),
 ('국가 차원', 'GUGU')]

In [232]:
import numpy as np
import math
vocab_len = len(nodes)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.matrix([[0 for i in range(len(vo_idx(nodes).vo_to_dic()))] for i in range(len(vo_idx(nodes).vo_to_dic()))])

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((weighted_edge.shape[0],1))

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 10

# weighted_edge 구현


weighted_edge

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [233]:
covered_coocurrences = weighted_edge
covered_coocurrences

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [234]:
def cc_maker(token):
  for idx, tok in enumerate(token):
    if window_size-1>0:
      if len(token[idx:])>=window_size:
        for win in range(1,window_size):
          if (tok in list(vo_idx(nodes).vo_to_dic().keys()))&(token[idx+win] in list(vo_idx(nodes).vo_to_dic().keys())):
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[(tok)],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1
      else:
        for win in range(1,len(token[idx:])):
          if (tok in list(vo_idx(nodes).vo_to_dic().keys()))&(token[idx+win] in list(vo_idx(nodes).vo_to_dic().keys())):
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[(tok)],vo_idx(nodes).vo_to_dic()[token[idx+win]]]=1
            covered_coocurrences[vo_idx(nodes).vo_to_dic()[token[idx+win]],vo_idx(nodes).vo_to_dic()[tok]]=1

In [235]:
cc_maker(tokens)

In [236]:
weighted_edge=covered_coocurrences/np.matrix([cc.sum() for cc in covered_coocurrences]).transpose()

In [237]:
weighted_edge

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.33333333],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.2       , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

### 3) 각 노드의 score계산

In [238]:
constant_edge=np.copy(weighted_edge)

In [239]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    for i in range(len(weighted_edge)):
      score[i]=(1-d)+d*((weighted_edge[:,i]).sum())
    for prev, curr in zip(prev_score,score):
      if abs(prev-curr)<threshold:
        break
    for j in range(len(weighted_edge)):
      weighted_edge[j,:]=constant_edge[j,:]*float(score[j])

In [240]:
weighted_edge

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.18241069],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.16519941, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [242]:
sorted_index = list(vo_idx(nodes).vo_to_dic().values())

n = 10

print("\n=== 핵심 구 ===")
for i in range(0,n):
    print(str(vo_idx(nodes).idx_to_dic()[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심 구 ===
('유영민 장관 등 참석', 'GUGU') : [0.54723207]
('산업 창출', 'GUGU') : [0.85143404]
('양질', 'GUGU') : [0.76005806]
('소비', 'GUGU') : [1.34208918]
('데이터 생태', 'GUGU') : [0.73714591]
('폐업', 'GUGU') : [1.02419829]
('헬스 케어', 'GUGU') : [0.80731689]
('데이터 부족 문제', 'GUGU') : [0.79174403]
('배포 금지', 'GUGU') : [0.78141859]
('분야', 'GUGU') : [2.02665463]
